In [ ]:
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random
import matplotlib.dates as mdates

# ML Imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
##########################
### DATA PREPROCESSING ###
##########################

chest12 = [
    [0.242148187,14.14903366],
    [1.072113351,13.53781202],
    [1.984894994,12.92633834],
    [3.02226143,12.19274636],
    [19.829146,-0.21492677],
    [20.98461594,1.120687017],
    [22.17969376,3.06487591],
    [22.63158369,4.280891897],
    [29.38454752,17.10381766],
    [30.0899279	,16.61471433],
    [31.00270954,16.00324064],
    [32.08130418,15.33039275],
    [33.07726238,14.59692679],
    [33.69514533,15.69069845],
]

rightlow12 = [
    [1.092637435,	20.59861913],
    [0.059771895,	19.81045828],
    [2.043046566,	21.26529179],
    [3.033783723,	22.29705911],
    [4.02578113,	22.90273564],
    [5.060447029,	23.08219536],
    [19.74146855,	1.428818335],
    [29.22125503,	16.31301029],
    [30.17130409,	17.10142318],
    [31.04123721,	16.97703643],
    [32.16106005,	16.36493262],
    [33.07150123,	16.54477041],
    [33.81666952	,16.60337207],
]

leftab12 = [
    [-0.472233975,17.68164265],
    [19.69717974	,2.40286617],
    [20.23170611	,3.679500221],
    [21.09605812	,5.442086975],
    [22.08445481	,7.265165768],
    [22.78713464	,7.689114134],
    [29.9316764	, 14.11954379],
    [31.09326756	,13.38557373],
    [32.04619719	,13.20006481],
    [33.16115907	,14.23145406],
]

rightab12 = [
    [5.306916076,	9.751010451],
    [19.82500518,	1.185085832],
    [21.02242346,	2.337963255],
    [22.05366868,	3.673955117],
    [29.35502165,	13.08651622],
    [30.01665331,	13.38885038],
    [31.01351169,	12.35103386],
    [32.17402263,	11.98228447],
    [33.04557607,	11.3100667],
    [33.62421122,	11.67352303],
]

back12 = [
    [-0.011882365,	16.03676332],
    [1.023863749,	15.85100235],
    [2.06014997	,15.48263104],
    [3.013439674,	15.1753819],
    [4.007957584,	14.92887685],
    [5.00517603	,13.76932009],
    [6.00095419	,13.09672425],
    [19.44476951,	3.74276481],
    [20.01944387	,5.445363627],
    [21.05194934	,6.355264698],
    [22.08931577	,5.621672713],
    [22.66831099,	5.863388814],
    [31.12855458	,15.45503155],
    [32.03881572	,15.69573945],
    [33.03279353	,15.63184474],
    [33.77760174	,15.81218663],
]

leftlow12 = [
    [1.01072113,	20.2945206],
    [2.04790753,	19.6217987],
    [3.08419375,	19.2534274],
    [3.95448694,	19.0073005 ],  
    [19.693579	,3.62026843],
    [20.1874173	,4.65354806],
    [21.1317052	,7.38980457],
    [21.9967774	,8.90891087],
    [22.7795731,	10.2456589  ],
    [29.502651	,19.1730234],
    [30.0393378	,19.7192161],
    [31.0333156,	19.6553214],
    [32.0708621,	18.8608593],
    [33.0626795,	19.527406],
    [33.5995463,	20.0127285],
]

leftup12 = [
    [0.013502687,	21.45407736],
    [1.007120417,	21.51192287],
    [2.043226602,	21.20442168],
    [3.037924547,	20.89704651],
    [4.031362241,	21.01576214],
    [5.028580688,	19.85620538],
    [19.24456967,	1.430330636],
    [19.94292864,	3.315161717],
    [21.01540207,	4.711897668],
    [22.04700735,	5.926149304],
    [22.83124342,	6.775936411],
    [29.4286563,	16.19063994],
    [30.04815958,	16.73658058],
    [31.00216943,	16.18585098],
    [32.03827562,	15.87834979],
    [33.11524993,	15.75333291],
    [34.19330447,	15.26309536],
]

rightup12 = [
    [1.011081205	,20.17278038],
    [2.004698935	,20.23062589],
    [2.997956593	,20.41021163],
    [4.031362241	,21.01576214],
    [5.066568247	,21.01261151],
    [5.980970213	,19.85330681],
    [19.16193322	,1.369712573],
    [19.89990008	,3.863118761],
    [20.97219347,5.320724824],
    [22.08103413	,8.421697918],
    [29.34349936	,16.98220346],
    [29.92105429	,17.71088047],
    [31.000009	,16.91629234],
    [32.07500293,17.46084671],
    [33.02991295,16.60576655],
    [34.02389075	,16.54187183],
]

core12 = [
    [1.125619835	,37.90671642],
    [2.180165289	,37.92537313],
    [3.138842975	,37.92537313],
    [4.19338843	,37.96268657],
    [5.056198347	,38],
    [6.014876033	,38.03731343],
    [7.165289256	,38.01865672],
    [8.123966942	,38.01865672],
    [8.795041322	,38],
    [10.04132231	,38],
    [10.90413223	,38.01865672],
    [12.05454545	,38.01865672],
    [13.01322314	,38],
    [14.0677686	,38.03731343],
    [15.02644628	,38.01865672],
    [15.98512397	,37.98134328],
    [17.03966942	,37.96268657],
    [17.90247934	,37.90671642],
    [19.05289256	,37.86940299],
    [20.01157025	,37.83208955],
    [21.0661157	,37.83208955],
    [22.12066116	,37.79477612],
    [23.17520661	,37.79477612],
    [24.03801653	,37.79477612],
    [24.99669421	,37.72014925],
    [25.9553719	,37.70149254],
    [26.91404959	,37.58955224],
    [28.06446281	,37.55223881],
    [29.0231405	,37.49626866],
    [29.98181818	,37.44029851],
    [31.1322314	,37.42164179],
    [31.99504132	,37.34701493],
    [32.95371901	,37.30970149],
    [34.00826446	,37.25373134],
    [35.15867769	,37.17910448],
]



In [ ]:
ind12 = list(range(0, 35))
chest12lin    = np.array(np.interp(ind12, [i[0] for i in chest12], [i[1] for i in chest12]))
leftab12lin   = np.array(np.interp(ind12, [i[0] for i in leftab12], [i[1] for i in leftab12]))
rightab12lin  = np.array(np.interp(ind12, [i[0] for i in rightab12], [i[1] for i in rightab12]))
back12lin     = np.array(np.interp(ind12, [i[0] for i in back12], [i[1] for i in back12]))
leftlow12lin  = np.array(np.interp(ind12, [i[0] for i in leftlow12], [i[1] for i in leftlow12]))
leftup12lin   = np.array(np.interp(ind12, [i[0] for i in leftup12], [i[1] for i in leftup12]))
rightup12lin  = np.array(np.interp(ind12, [i[0] for i in rightup12], [i[1] for i in rightup12]))
rightlow12lin = np.array(np.interp(ind12, [i[0] for i in rightlow12], [i[1] for i in rightlow12]))
core12lin     = np.array(np.interp(ind12, [i[0] for i in core12], [i[1] for i in core12]))

chest12grad = np.array(np.gradient(chest12lin))
leftab12grad = np.array(np.gradient(leftab12lin))
rightab12grad = np.array(np.gradient(rightab12lin))
back12grad = np.array(np.gradient(back12lin))
leftlow12grad = np.array(np.gradient(leftlow12lin))
leftup12grad = np.array(np.gradient(leftup12lin))
rightup12grad = np.array(np.gradient(rightup12lin))
rightlow12grad = np.array(np.gradient(rightlow12lin))
core12grad = np.array(np.gradient(core12lin))

In [ ]:
# plt.plot(ind12, chest12lin, 'r')
# plt.plot(ind12, leftab12lin, 'g')
# plt.plot(ind12, rightab12lin, 'b')
# plt.plot(ind12, back12lin, 'y')
# plt.plot(ind12, leftlow12lin, 'c')
# plt.plot(ind12, leftup12lin, 'm')
# plt.plot(ind12, rightup12lin, 'k')
# plt.plot(ind12, rightlow12lin, '#4fa8e4')
plt.plot(ind12, core12lin, '#000000')

# plt.plot(ind12, chest12grad, 'r')
# plt.plot(ind12, leftab12grad, 'g')
# plt.plot(ind12, rightab12grad, 'b')
# plt.plot(ind12, back12grad, 'y')
# plt.plot(ind12, leftlow12grad, 'c')
# plt.plot(ind12, leftup12grad, 'm')
# plt.plot(ind12, rightup12grad, 'k')
# plt.plot(ind12, rightlow12grad, '#4fa8e4')
# plt.plot(ind12, core12grad, '#000000', marker=".")
